In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

from src.utils import convert_bytes_to_megabytes, convert_milliseconds_to_minute, convert_bytes_to_giga_bytes, \
    convert_megabytes_to_gigabytes
from scripts.load_data import load_data_using_sqlalchemy

In [ ]:
query = "SELECT * FROM cleaned_xdr_data"
df = load_data_using_sqlalchemy(query)

df.head()

In [ ]:
df.columns

In [ ]:
df.dtypes

In [ ]:
df.rename(columns={"MSISDN/Number": "MSISDN"}, inplace=True)
df.columns

In [ ]:
df["Total Data Usage (Bytes)"] = df["Total DL (Bytes)"] + df["Total UL (Bytes)"]
df.head()

In [ ]:
users_engagement = df.groupby("MSISDN").agg({
    "Bearer Id": ["count"],
    "Dur. (ms)": ["sum"],
    "Total Data Usage (Bytes)": ["sum"]
}).reset_index()

users_engagement.columns = ["MSISDN", "Sessions Frequency", "Total Duration Session (ms)", "Total Data Usage (Bytes)"]
users_engagement.head()

In [ ]:
convert_bytes_to_megabytes(df, ["Total Data Usage (Bytes)"])
users_engagement.rename(columns={"Total Data Usage (Bytes)": "Total Data Usage (MB)"}, inplace=True)

In [ ]:
users_engagement.describe()

In [ ]:
top_10_sessions_frequency = users_engagement.sort_values(by="Sessions Frequency", ascending=False, axis=0).nlargest(10,
                                                                                                                    "Sessions Frequency")
convert_milliseconds_to_minute(top_10_sessions_frequency, ["Total Duration Session (ms)"])
convert_megabytes_to_gigabytes(top_10_sessions_frequency, ["Total Data Usage (MB)"])

top_10_sessions_frequency.rename(columns={"Total Duration Session (ms)": "Total Duration Session (min)",
                                          "Total Data Usage (MB)": "Total Data Usage (GB)"}, inplace=True)
top_10_sessions_frequency.head(10)

In [ ]:
top_10_sessions_duration = users_engagement.sort_values(by="Total Duration Session (ms)", ascending=False,
                                                        axis=0).nlargest(10, "Total Duration Session (ms)")

convert_milliseconds_to_minute(top_10_sessions_duration, ["Total Duration Session (ms)"])
convert_megabytes_to_gigabytes(top_10_sessions_duration, ["Total Data Usage (MB)"])

top_10_sessions_duration.rename(columns={"Total Duration Session (ms)": "Total Duration Session (min)",
                                         "Total Data Usage (MB)": "Total Data Usage (GB)"}, inplace=True)
top_10_sessions_duration.head(10)

In [ ]:
top_10_data_usage = users_engagement.sort_values(by="Total Data Usage (MB)", ascending=False, axis=0).nlargest(10,
                                                                                                               "Total Data Usage (MB)")

convert_milliseconds_to_minute(top_10_data_usage, ["Total Duration Session (ms)"])
convert_megabytes_to_gigabytes(top_10_data_usage, ["Total Data Usage (MB)"])

top_10_data_usage.rename(columns={"Total Duration Session (ms)": "Total Duration Session (min)",
                                  "Total Data Usage (MB)": "Total Data Usage (GB)"}, inplace=True)
top_10_data_usage.head(10)

In [ ]:
scaler = StandardScaler()

normalized_data = scaler.fit_transform(users_engagement)
normalized_df = pd.DataFrame(normalized_data, columns=["MSISDN", "Sessions Frequency", "Total Duration Session (ms)",
                                                       "Total Data Usage (Bytes)"])

normalized_df.head()

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=42)
cluster_labels = kmeans.fit_predict(normalized_df)

users_engagement["Cluster"] = cluster_labels
users_engagement.head()

In [ ]:
cluster_stats = users_engagement.groupby("Cluster").agg({
    "Sessions Frequency": ["min", "max", "mean", "sum"],
    "Total Duration Session (ms)": ["min", "max", "mean", "sum"],
    "Total Data Usage (MB)": ["min", "max", "mean", "sum"],
}).reset_index()

cluster_stats.columns = ["Cluster", "Min Sessions Frequency", "Max Sessions Frequency", "Mean Sessions Frequency",
                         "Sum Sessions Frequency", "Min Total Duration Session (ms)", "Max Total Duration Session (ms)",
                         "Mean Total Duration Session (ms)", "Sum Total Duration Session (ms)",
                         "Min Total Data Usage (MB)",
                         "Max Total Data Usage (MB)", "Mean Total Data Usage (MB)", "Sum Total Data Usage (MB)"
                         ]

convert_megabytes_to_gigabytes(cluster_stats, ["Min Total Data Usage (MB)", "Max Total Data Usage (MB)",
                                               "Mean Total Data Usage (MB)", "Sum Total Data Usage (MB)"])

convert_milliseconds_to_minute(cluster_stats, ["Min Total Duration Session (ms)", "Max Total Duration Session (ms)",
                                               "Mean Total Duration Session (ms)", "Sum Total Duration Session (ms)"])
cluster_stats.rename(columns={"Min Total Duration Session (ms)": "Min Total Duration Session (min)",
                              "Max Total Duration Session (ms)": "Max Total Duration Session (min)",
                              "Mean Total Duration Session (ms)": "Mean Total Duration Session (min)",
                              "Sum Total Duration Session (ms)": "Sum Total Duration Session (min)"}, inplace=True)

cluster_stats.rename(columns={"Min Total Data Usage (MB)": "Min Total Data Usage (GB)",
                              "Max Total Data Usage (MB)": "Max Total Data Usage (GB)",
                              "Mean Total Data Usage (MB)": "Mean Total Data Usage (GB)",
                              "Sum Total Data Usage (MB)": "Sum Total Data Usage (GB)"}, inplace=True)

cluster_stats.head()

### Mean Metrics per Cluster

In [ ]:
metrics = ['Sessions Frequency', 'Total Duration Session (min)', 'Total Data Usage (GB)']
x = np.arange(len(metrics))
width = 0.2

fig, ax = plt.subplots(figsize=(20, 10))

for i, cluster in enumerate(cluster_stats['Cluster']):
    means = cluster_stats.loc[cluster_stats['Cluster'] == cluster,
    ['Mean Sessions Frequency', 'Mean Total Duration Session (min)', 'Mean Total Data Usage (GB)']]
    ax.bar(x + i * width, means.values[0], width, label=f'Cluster {cluster}')

ax.set_ylabel('Mean Values')
ax.set_title('Mean Metrics by Cluster')
ax.set_xticks(x + width)
ax.set_xticklabels(metrics)
ax.legend()

ax.set_yscale('log')

plt.tight_layout()
plt.show()

### Minimum Metrics per Cluster

In [ ]:
metrics = ['Sessions Frequency', 'Total Duration Session (min)', 'Total Data Usage (GB)']
x = np.arange(len(metrics))
width = 0.2

fig, ax = plt.subplots(figsize=(20, 10))

for i, cluster in enumerate(cluster_stats['Cluster']):
    values = cluster_stats.loc[cluster_stats['Cluster'] == cluster,
    [f'Min {metric}' for metric in metrics]]
    ax.bar(x + i * width, values.values[0], width, label=f'Cluster {cluster}')

ax.set_ylabel('Minimum Values')
ax.set_title('Minimum Metrics by Cluster')
ax.set_xticks(x + width)
ax.set_xticklabels(metrics)
ax.legend()
ax.set_yscale('log')

plt.tight_layout()
plt.show()

### Maximum Metrics per Cluster

In [ ]:
metrics = ['Sessions Frequency', 'Total Duration Session (min)', 'Total Data Usage (GB)']
x = np.arange(len(metrics))
width = 0.2

fig, ax = plt.subplots(figsize=(20, 10))

for i, cluster in enumerate(cluster_stats['Cluster']):
    values = cluster_stats.loc[cluster_stats['Cluster'] == cluster,
    [f'Max {metric}' for metric in metrics]]
    ax.bar(x + i * width, values.values[0], width, label=f'Cluster {cluster}')

ax.set_ylabel('Maximum Values')
ax.set_title('Maximum Metrics by Cluster')
ax.set_xticks(x + width)
ax.set_xticklabels(metrics)
ax.legend()
ax.set_yscale('log')

plt.tight_layout()
plt.show()

### Sum Metrics per Cluster

In [ ]:
metrics = ['Sessions Frequency', 'Total Duration Session (min)', 'Total Data Usage (GB)']
x = np.arange(len(metrics))
width = 0.2

fig, ax = plt.subplots(figsize=(20, 10))

for i, cluster in enumerate(cluster_stats['Cluster']):
    values = cluster_stats.loc[cluster_stats['Cluster'] == cluster,
    [f'Sum {metric}' for metric in metrics]]
    ax.bar(x + i * width, values.values[0], width, label=f'Cluster {cluster}')

ax.set_ylabel('Sum Values')
ax.set_title('Sum Metrics by Cluster')
ax.set_xticks(x + width)
ax.set_xticklabels(metrics)
ax.legend()
ax.set_yscale('log')

plt.tight_layout()
plt.show()

### Application Traffic by User Analysis

In [189]:
df.columns

Index(['Bearer Id', 'Start', 'End', 'IMSI', 'MSISDN', 'IMEI',
       'Last Location Name', 'Avg RTT DL (ms)', 'Avg RTT UL (ms)',
       'Avg Bearer TP DL (kbps)', 'Avg Bearer TP UL (kbps)',
       'TCP DL Retrans. Vol (Bytes)', 'TCP UL Retrans. Vol (Bytes)',
       'DL TP < 50 Kbps (%)', '50 Kbps < DL TP < 250 Kbps (%)',
       '250 Kbps < DL TP < 1 Mbps (%)', 'DL TP > 1 Mbps (%)',
       'UL TP < 10 Kbps (%)', '10 Kbps < UL TP < 50 Kbps (%)',
       '50 Kbps < UL TP < 300 Kbps (%)', 'UL TP > 300 Kbps (%)',
       'HTTP DL (Bytes)', 'HTTP UL (Bytes)', 'Activity Duration DL (ms)',
       'Activity Duration UL (ms)', 'Dur. (ms)', 'Handset Manufacturer',
       'Handset Type', 'Nb of sec with 125000B < Vol DL',
       'Nb of sec with 1250B < Vol UL < 6250B',
       'Nb of sec with 31250B < Vol DL < 125000B',
       'Nb of sec with 37500B < Vol UL',
       'Nb of sec with 6250B < Vol DL < 31250B',
       'Nb of sec with 6250B < Vol UL < 37500B',
       'Nb of sec with Vol DL < 6250B', 'Nb 

In [190]:
df_copy = df.copy()
df_copy.head()

,Bearer Id,Start,End,IMSI,MSISDN,IMEI,Last Location Name,Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),...,Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes),Total Data Usage (Bytes)
0,1.311448e+19,2019-04-04 12:01:00.770,2019-04-25 14:35:00.662,208201448079117,33664962239,35521209507511,9.16456699548519E+015,42.000000,5.000000,23.0,...,2501332.0,8198936.0,9656251.0,278082303.0,14344150.0,171744450.0,8814393.0,36749741.0,308879636.0,345.629377
1,1.311448e+19,2019-04-09 13:04:00.235,2019-04-25 08:15:00.606,208201909211140,33681854413,35794009006359,L77566A,65.000000,5.000000,16.0,...,19111729.0,18338413.0,17227132.0,608750074.0,1170709.0,526904238.0,15055145.0,53800391.0,653384965.0,707.185356
2,1.311448e+19,2019-04-09 17:42:00.001,2019-04-25 11:58:00.652,208200314458056,33760627129,35281510359387,D42335A,105.958376,17.633438,6.0,...,14699576.0,17587794.0,6163408.0,229584621.0,395630.0,410692588.0,4215763.0,27883638.0,279807335.0,307.690973
3,1.311448e+19,2019-04-10 00:31:00.486,2019-04-25 07:36:00.171,208201402342131,33750343200,35356610164913,T21824A,105.958376,17.633438,44.0,...,15146643.0,13994646.0,1097942.0,799538153.0,10849722.0,749039933.0,12797283.0,43324218.0,846028530.0,889.352748
4,1.311448e+19,2019-04-12 20:10:00.565,2019-04-25 10:40:00.954,208201401415120,33699795932,35407009745539,D88865A,105.958376,17.633438,6.0,...,18962873.0,17124581.0,415218.0,527707248.0,3529801.0,550709500.0,13910322.0,38542814.0,569138589.0,607.681403


In [191]:
df_copy["Youtube Data Usage (Bytes)"] = df_copy["Youtube DL (Bytes)"] + df_copy["Youtube UL (Bytes)"]
df_copy["Social Media Data Usage (Bytes)"] = df_copy["Social Media DL (Bytes)"] + df_copy["Social Media UL (Bytes)"]
df_copy["Netflix Data Usage (Bytes)"] = df_copy["Netflix DL (Bytes)"] + df_copy["Netflix UL (Bytes)"]
df_copy["Google Data Usage (Bytes)"] = df_copy["Google DL (Bytes)"] + df_copy["Google UL (Bytes)"]
df_copy["Email Data Usage (Bytes)"] = df_copy["Email DL (Bytes)"] + df_copy["Email UL (Bytes)"]
df_copy["Gaming Data Usage (Bytes)"] = df_copy["Gaming DL (Bytes)"] + df_copy["Gaming UL (Bytes)"]
df_copy["Other Data Usage (Bytes)"] = df_copy["Other DL (Bytes)"] + df_copy["Other UL (Bytes)"]

df_copy.drop(labels=["Social Media DL (Bytes)", "Youtube DL (Bytes)", "Netflix DL (Bytes)", "Google DL (Bytes)",
                     "Email DL (Bytes)", "Gaming DL (Bytes)", "Other DL (Bytes)", "Social Media UL (Bytes)",
                     "Youtube UL (Bytes)", "Netflix UL (Bytes)", "Google UL (Bytes)", "Email UL (Bytes)",
                     "Gaming UL (Bytes)", "Other UL (Bytes)"], axis=1, inplace=True)

df_copy.head(10)

,Bearer Id,Start,End,IMSI,MSISDN,IMEI,Last Location Name,Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),...,Total UL (Bytes),Total DL (Bytes),Total Data Usage (Bytes),Youtube Data Usage (Bytes),Social Media Data Usage (Bytes),Netflix Data Usage (Bytes),Google Data Usage (Bytes),Email Data Usage (Bytes),Gaming Data Usage (Bytes),Other Data Usage (Bytes)
0,1.311448e+19,2019-04-04 12:01:00.770,2019-04-25 14:35:00.662,208201448079117,33664962239,35521209507511,9.16456699548519E+015,42.000000,5.000000,23.0,...,36749741.0,308879636.0,345.629377,18355943.0,1570185.0,17855187.0,2905912.0,3701304.0,292426453.0,180558843.0
1,1.311448e+19,2019-04-09 13:04:00.235,2019-04-25 08:15:00.606,208201909211140,33681854413,35794009006359,L77566A,65.000000,5.000000,16.0,...,53800391.0,653384965.0,707.185356,39359124.0,1933278.0,35565545.0,4414096.0,937385.0,609920783.0,541959383.0
2,1.311448e+19,2019-04-09 17:42:00.001,2019-04-25 11:58:00.652,208200314458056,33760627129,35281510359387,D42335A,105.958376,17.633438,6.0,...,27883638.0,279807335.0,307.690973,34425237.0,1726277.0,23751202.0,10229119.0,3363124.0,229980251.0,414908351.0
3,1.311448e+19,2019-04-10 00:31:00.486,2019-04-25 07:36:00.171,208201402342131,33750343200,35356610164913,T21824A,105.958376,17.633438,44.0,...,43324218.0,846028530.0,889.352748,36534765.0,657493.0,15092588.0,11811761.0,2070983.0,810387875.0,761837216.0
4,1.311448e+19,2019-04-12 20:10:00.565,2019-04-25 10:40:00.954,208201401415120,33699795932,35407009745539,D88865A,105.958376,17.633438,6.0,...,38542814.0,569138589.0,607.681403,34222253.0,912788.0,17539799.0,7748843.0,2110349.0,531237049.0,564619822.0
5,1.311448e+19,2019-04-12 21:37:00.439,2019-04-25 08:08:00.553,208201402670191,33668185951,35298410295700,T89132C,105.958376,17.633438,70.0,...,30307754.0,754452212.0,784.759966,23944031.0,3200014.0,13881809.0,12934133.0,2902410.0,723461512.0,659414334.0
6,1.311448e+19,2019-04-13 08:41:00.612,2019-04-25 08:16:00.168,208201448341047,33665368271,86762704322800,9.16456701058919E+015,102.000000,5.000000,22.0,...,47925246.0,70562047.0,118.487293,9120760.0,1624991.0,31212766.0,4906872.0,2090979.0,58881157.0,814303325.0
7,1.304243e+19,2019-04-14 02:11:00.592,2019-04-25 02:26:00.512,208201009069207,33763490140,86546404964017,CELL_208_20_520025_1,39.000000,18.000000,3698.0,...,58813016.0,775350343.0,834.163359,31924969.0,3161403.0,18933702.0,3872713.0,3500104.0,765055811.0,545316477.0
8,1.311448e+19,2019-04-14 12:48:00.121,2019-04-25 10:22:00.960,208201448324028,33698743617,35562409696153,T42084A,105.958376,17.633438,46.0,...,42363146.0,861612261.0,903.975407,29297508.0,372625.0,19054441.0,8902099.0,3380069.0,828219634.0,187669517.0
9,1.304243e+19,2019-04-15 00:32:00.000,2019-04-25 00:40:00.284,208200314385130,33659219748,35573109931422,L20434C,97.000000,7.000000,3845.0,...,22417975.0,850570347.0,872.988322,22911539.0,1585413.0,13185350.0,8778341.0,766046.0,812946425.0,77671250.0


In [192]:
applications = ["Youtube", "Social Media", "Netflix", "Google", "Email", "Gaming", "Other"]
cols = [f"{app} Data Usage (Bytes)" for app in applications]

df_copy = df_copy[cols + ["MSISDN"]]
df_copy

,Youtube Data Usage (Bytes),Social Media Data Usage (Bytes),Netflix Data Usage (Bytes),Google Data Usage (Bytes),Email Data Usage (Bytes),Gaming Data Usage (Bytes),Other Data Usage (Bytes),MSISDN
0,18355943.0,1570185.0,17855187.0,2905912.0,3701304.0,292426453.0,180558843.0,33664962239
1,39359124.0,1933278.0,35565545.0,4414096.0,937385.0,609920783.0,541959383.0,33681854413
2,34425237.0,1726277.0,23751202.0,10229119.0,3363124.0,229980251.0,414908351.0,33760627129
3,36534765.0,657493.0,15092588.0,11811761.0,2070983.0,810387875.0,761837216.0,33750343200
4,34222253.0,912788.0,17539799.0,7748843.0,2110349.0,531237049.0,564619822.0,33699795932
...,...,...,...,...,...,...,...,...
148341,24553645.0,993175.0,29618711.0,1265634.0,4280107.0,797160102.0,336273648.0,33668648496
148342,27955095.0,3517065.0,37561864.0,12784914.0,690876.0,535806880.0,16751926.0,33650688697
148343,22165518.0,2352181.0,40643294.0,4415361.0,2121718.0,631628095.0,714638145.0,33663449963
148344,24516413.0,1260239.0,15029702.0,6585469.0,2058277.0,566933800.0,132415585.0,33621890103


In [193]:
convert_bytes_to_megabytes(df_copy, cols)

df_copy.columns = [f"{app} Data Usage (MB)" for app in applications] + ["MSISDN"]
df_copy.index = df_copy["MSISDN"]
df_copy.index.name = None
df_copy.head(10)

,Youtube Data Usage (MB),Social Media Data Usage (MB),Netflix Data Usage (MB),Google Data Usage (MB),Email Data Usage (MB),Gaming Data Usage (MB),Other Data Usage (MB),MSISDN
33664962239,18.355943,1.570185,17.855187,2.905912,3.701304,292.426453,180.558843,33664962239
33681854413,39.359124,1.933278,35.565545,4.414096,0.937385,609.920783,541.959383,33681854413
33760627129,34.425237,1.726277,23.751202,10.229119,3.363124,229.980251,414.908351,33760627129
33750343200,36.534765,0.657493,15.092588,11.811761,2.070983,810.387875,761.837216,33750343200
33699795932,34.222253,0.912788,17.539799,7.748843,2.110349,531.237049,564.619822,33699795932
33668185951,23.944031,3.200014,13.881809,12.934133,2.902410,723.461512,659.414334,33668185951
33665368271,9.120760,1.624991,31.212766,4.906872,2.090979,58.881157,814.303325,33665368271
33763490140,31.924969,3.161403,18.933702,3.872713,3.500104,765.055811,545.316477,33763490140
33698743617,29.297508,0.372625,19.054441,8.902099,3.380069,828.219634,187.669517,33698743617
33659219748,22.911539,1.585413,13.185350,8.778341,0.766046,812.946425,77.671250,33659219748


In [194]:
df_copy.columns

Index(['Youtube Data Usage (MB)', 'Social Media Data Usage (MB)',
       'Netflix Data Usage (MB)', 'Google Data Usage (MB)',
       'Email Data Usage (MB)', 'Gaming Data Usage (MB)',
       'Other Data Usage (MB)', 'MSISDN'],
      dtype='object')

In [195]:
grouped_data = df_copy.groupby('MSISDN').sum().reset_index()
grouped_data.head()

,MSISDN,Youtube Data Usage (MB),Social Media Data Usage (MB),Netflix Data Usage (MB),Google Data Usage (MB),Email Data Usage (MB),Gaming Data Usage (MB),Other Data Usage (MB)
0,3197020876596,11.959905,0.715224,26.592300,10.438660,1.520771,178.048738,470.526473
1,33601001722,21.624548,2.232135,27.180981,4.389005,1.331362,812.458661,386.570872
2,33601001754,12.432223,2.660565,11.221763,5.334863,3.307781,119.750078,281.710071
3,33601002511,21.333570,3.195623,19.353900,3.443126,3.205380,538.827713,501.693672
4,33601007832,6.977321,0.280294,1.942092,9.678493,2.284670,391.126127,35.279702


In [200]:
youtube_10_users = grouped_data.sort_values(by="Youtube Data Usage (MB)", axis=0, ascending=False).nlargest(10, "Youtube Data Usage (MB)")
youtube_10_users = youtube_10_users["MSISDN"]

youtube_10_users

13108    33625779332
92488    33760536639
6395     33614892860
13452    33626320676
76000    33675877202
64826    33667163239
13919    33627080969
92143    33760413819
86046    33698792269
657      33603127838
Name: MSISDN, dtype: object

In [201]:
socials_10_users = grouped_data.sort_values(by="Social Media Data Usage (MB)", axis=0, ascending=False).nlargest(10, "Social Media Data Usage (MB)")
socials_10_users = socials_10_users["MSISDN"]

socials_10_users

13452     33626320676
92488     33760536639
36868     33659725664
6395      33614892860
13108     33625779332
64826     33667163239
105640    33786323068
70636     33669068942
657       33603127838
60068     33665530316
Name: MSISDN, dtype: object

In [204]:
netflix_10_users = grouped_data.sort_values(by="Netflix Data Usage (MB)", axis=0, ascending=False).nlargest(10, "Netflix Data Usage (MB)")
netflix_10_users = netflix_10_users["MSISDN"]

netflix_10_users

36868     33659725664
6395      33614892860
13108     33625779332
92488     33760536639
13452     33626320676
92143     33760413819
64826     33667163239
76000     33675877202
105640    33786323068
94210     33761268199
Name: MSISDN, dtype: object

In [205]:
email_10_users = grouped_data.sort_values(by="Email Data Usage (MB)", axis=0, ascending=False).nlargest(10, "Email Data Usage (MB)")
email_10_users = email_10_users["MSISDN"]

email_10_users

13452     33626320676
6395      33614892860
13108     33625779332
105640    33786323068
36868     33659725664
92488     33760536639
76000     33675877202
59817     33665460546
64826     33667163239
86046     33698792269
Name: MSISDN, dtype: object

In [206]:
gaming_10_users = grouped_data.sort_values(by="Gaming Data Usage (MB)", axis=0, ascending=False).nlargest(10, "Gaming Data Usage (MB)")
gaming_10_users = gaming_10_users["MSISDN"]

gaming_10_users

6395     33614892860
92488    33760536639
13108    33625779332
76000    33675877202
13452    33626320676
36868    33659725664
62747    33666464084
92143    33760413819
56978    33664712899
86046    33698792269
Name: MSISDN, dtype: object

In [202]:
other_10_users = grouped_data.sort_values(by="Other Data Usage (MB)", axis=0, ascending=False).nlargest(10, "Other Data Usage (MB)")
other_10_users = other_10_users["MSISDN"]

other_10_users

13452    33626320676
6395     33614892860
76000    33675877202
13108    33625779332
657      33603127838
36868    33659725664
13862    33626948251
13919    33627080969
94210    33761268199
30556    33658361927
Name: MSISDN, dtype: object